# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770990016815                   -0.53    9.0         
  2   -2.772150966751       -2.94       -1.32    1.0    105ms
  3   -2.772170360745       -4.71       -2.51    1.0    112ms
  4   -2.772170658209       -6.53       -3.27    1.0    129ms
  5   -2.772170721955       -7.20       -3.95    2.0    123ms
  6   -2.772170722623       -9.18       -4.22    1.0    131ms
  7   -2.772170722998       -9.43       -4.94    1.0    113ms
  8   -2.772170723015      -10.78       -5.73    2.0    131ms
  9   -2.772170723015      -12.27       -6.42    1.0    127ms
 10   -2.772170723015   +  -13.72       -6.44    2.0    142ms
 11   -2.772170723015      -13.57       -7.14    1.0    138ms
 12   -2.772170723015      -13.95       -7.45    1.0    119ms
 13   -2.772170723015   +  -14.07       -8.27    2.0    156ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ----

1.773558119098397

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770777785368                   -0.53    9.0         
  2   -2.772058409558       -2.89       -1.32    1.0    105ms
  3   -2.772083282849       -4.60       -2.52    1.0    109ms
  4   -2.772083385093       -6.99       -3.38    1.0    144ms
  5   -2.772083415034       -7.52       -3.83    2.0    123ms
  6   -2.772083417737       -8.57       -4.97    1.0    137ms
  7   -2.772083417804      -10.17       -5.13    2.0    127ms
  8   -2.772083417811      -11.21       -6.22    1.0    116ms
  9   -2.772083417811      -13.23       -7.01    1.0    133ms
 10   -2.772083417811      -14.65       -7.54    2.0    138ms
 11   -2.772083417811      -15.05       -7.48    1.0    132ms
 12   -2.772083417811      -14.05       -7.74    1.0    116ms
 13   -2.772083417811      -14.31       -8.10    1.0    139ms

Polarizability via ForwardDiff:       1.7725349677310585
Polarizabilit